In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

global num
num=str(6)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.loadtxt('distance_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['distance_matrix']=np.dot(data['distance_matrix'],np.diag([100000000000000] * len(data['distance_matrix'])))
    demands=[]
    for i in np.loadtxt('demand_'+str(num)+'.txt', dtype=np.float32, delimiter=','):
        demands.append(int(i[1]))
    data['demands'] = demands
    fname = 'solution_'+str(num)+'.txt'
    with open(fname, 'r+', encoding='utf-8') as f:
        s = [i[:-1].split(',') for i in f.readlines()]
    for i in range(len(s)):
        for j in range(len(s[i])):
            q=s[i][j]
            s[i][j]=int(q)-1
        s[i].remove(0)
        s[i].remove(0)

    data['initial_routes']=s
    
    w=np.loadtxt('spec_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['vehicle_capacities'] = []
    data['num_vehicles'] = int(w[1])
    for j in range(int(w[1])):
        data['vehicle_capacities'].append(int(w[2]))
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    
    f=open("solution_"+num+".txt","w")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        l=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            l.append(int('{0}'.format(node_index, route_load)))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        for i in range(len(l)):
            l[i]=l[i]+1
        l.append(1)
        kk=0
        for i in l:
            kk+=1
            if kk==len(l):
                f.write(str(i))
            else:
                f.write(str(i)+',')
        f.write('\n')
        print(l)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    f.close()


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
#     initial_solution = routing.ReadAssignmentFromRoutes(data['initial_routes'],True)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC)
    search_parameters.time_limit.seconds = 300
    search_parameters.log_search = True
    search_parameters.use_full_propagation=True
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#     search_parameters.solution_limit = 50

    

#     # Solve the problem.
    initial_solution = routing.ReadAssignmentFromRoutes(data['initial_routes'],True)
    solution = routing.SolveFromAssignmentWithParameters(initial_solution,search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 7795580951118469005
[1, 973, 589, 347, 198, 212, 281, 195, 953, 641, 548, 851, 656, 928, 740, 584, 720, 649, 529, 500, 323, 826, 105, 49, 571, 232, 989, 1]
Route for vehicle 0:
 0 Load(0) ->  972 Load(3) ->  588 Load(10) ->  346 Load(17) ->  197 Load(24) ->  211 Load(27) ->  280 Load(34) ->  194 Load(36) ->  952 Load(43) ->  640 Load(53) ->  547 Load(55) ->  850 Load(62) ->  655 Load(64) ->  927 Load(70) ->  739 Load(73) ->  583 Load(78) ->  719 Load(81) ->  648 Load(88) ->  528 Load(95) ->  499 Load(102) ->  322 Load(111) ->  825 Load(121) ->  104 Load(125) ->  48 Load(126) ->  570 Load(129) ->  231 Load(130) ->  988 Load(131) ->  0 Load(131)
Distance of the route: 130404987072944634m
Load of the route: 131

[1, 549, 557, 531, 879, 320, 632, 667, 272, 337, 167, 277, 5, 987, 282, 528, 505, 701, 253, 427, 688, 504, 107, 47, 1]
Route for vehicle 1:
 0 Load(0) ->  548 Load(1) ->  556 Load(10) ->  530 Load(12) ->  878 Load(22) ->  319 Load(23) ->  631 Load(32) ->  666 Load(37) -

In [2]:
# 7900